Application of Once Class SVM to detect anomaly : https://www.kaggle.com/amarnayak/once-class-svm-to-detect-anomaly/data

Code qui compare une SVM sur R et Python : http://eric.univ-lyon2.fr/~ricco/tanagra/fichiers/fr_Tanagra_SVM_R_Python.pdf

Site pour comprendre ce qu'est une One-Class SVM : http://rvlasveld.github.io/blog/2013/07/12/introduction-to-one-class-support-vector-machines/

Fonction sur python de la fonction OneClassSVM (fonction équivalente à la section du site au-dessus de la One-Class SVM selon Schölkopf): https://scikit-learn.org/stable/modules/generated/sklearn.svm.OneClassSVM.html

Exemple utilisant la fonction One-class SVM : https://scikit-learn.org/stable/auto_examples/svm/plot_oneclass.html#sphx-glr-auto-examples-svm-plot-oneclass-py

In [3]:
import pandas as pd
import numpy as np
from sklearn import svm

In [4]:
data_train = pd.read_hdf('train.hdf5')

In [ ]:
data_train_Y = data_train

Les données sont composées de séquences d'une minute de mesure d'accéléromètre à une fréquence de 1024 Herts.

60 * 1024 = 61440 variables/colonnes

Training Data : Le jeux de données d'apprentissage est composé de 1677 séquences.

Toutes ces séquences sont considérées comme des séquences normales.

Elles doivent être utilisées pour apprendre le comportement normal des accéléromètres.

In [5]:
data_valid = pd.read_hdf('validation.hdf5')

Validation Data : Le jeu de données de validation est composé de 594 séquences.
    
Les localisations et les angles peuvent, ou pas, être différentes de celles du jeu de données d'apprentissage. 

Un taux inconnu de ces séquences présentens un comportement anormal. 

Ces séquences doivent être détectées en les comparant aux séquences normales du jeu de données d'apprentissage.

In [3]:
data_train

,0,1,2,3,4,5,6,7,8,9,...,61430,61431,61432,61433,61434,61435,61436,61437,61438,61439
0,0.041259,0.041259,0.032573,0.023887,0.029315,0.041259,0.045602,0.038001,0.030401,0.032573,...,0.034744,-0.007600,-0.003257,0.065145,0.047773,-0.009772,0.031487,0.096632,0.077089,0.040173
1,-0.211722,-0.264924,-0.274696,-0.236694,-0.156349,-0.059716,0.005429,0.046687,0.153091,0.281210,...,0.260581,-0.004343,-0.241037,-0.636252,-0.953292,-0.980436,-0.846888,-0.838202,-0.880546,-0.739398
2,0.214105,0.154930,0.136640,0.013987,-0.038733,-0.015063,-0.111894,-0.104363,0.047340,-0.054871,...,0.040884,0.375490,0.699337,0.965085,1.086662,1.132926,1.279249,1.296464,0.937112,0.451879
3,-0.154837,-0.127768,-0.217638,-0.284770,-0.299929,-0.270694,-0.077960,0.092036,0.076877,0.154837,...,-0.286936,-0.171079,-0.036814,-0.024904,0.031400,0.140761,-0.011911,-0.173244,-0.063884,0.081208
4,-1.022780,-0.916376,-0.676425,-0.461445,-0.330069,-0.122690,0.178064,0.489675,0.799115,0.931577,...,1.009751,1.134613,0.836030,0.479903,0.109661,-0.285553,-0.628651,-0.916376,-1.010837,-0.804544
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1672,-0.063989,0.017228,0.015997,-0.321177,-0.503300,-0.535295,-0.562367,-0.600515,-0.573442,-0.317485,...,-0.121826,-0.134131,-0.111981,-0.057836,-0.073834,-0.125517,-0.082448,-0.036917,-0.114442,-0.183354
1673,0.993037,0.726052,0.341885,0.286314,0.535177,0.605245,0.198124,-0.375711,-0.579875,-0.495310,...,0.038658,0.188460,0.273025,0.045907,-0.108727,0.072484,0.415577,0.356382,-0.096646,-0.280273
1674,0.570550,0.253578,-0.193841,-0.502279,-0.657108,-0.838757,-0.971642,-0.881427,-0.700996,-0.520566,...,-0.146295,0.128008,0.162143,0.018287,-0.017068,0.087777,0.062175,-0.041450,-0.140199,-0.247482
1675,0.630677,0.605010,0.477897,0.155225,-0.441229,-0.679566,-0.515786,-0.424118,-0.183336,0.394784,...,0.229781,-0.041556,-0.176003,-0.301894,-0.177225,0.033001,0.025667,0.088001,0.206559,0.003667


In [5]:
data_valid.head(5)

,0,1,2,3,4,5,6,7,8,9,...,61430,61431,61432,61433,61434,61435,61436,61437,61438,61439
0,0.070650,0.085524,0.076847,0.085524,0.120229,0.140061,0.107834,0.048340,0.014874,0.030987,...,0.018592,-0.017353,-0.049579,-0.061974,-0.022311,0.052058,0.091721,0.075608,0.045861,0.027268
1,-0.300685,-0.882656,-1.733789,-2.181179,-1.942328,-1.719240,-1.981126,-2.323034,-2.325459,-2.349708,...,0.454665,-0.782024,-1.725302,-1.829572,-1.170004,-0.278861,-0.152767,-1.035424,-2.312123,-3.199628
2,-0.056398,-0.152028,-0.058850,0.140994,0.170419,0.154480,0.270954,0.359228,0.334708,0.349420,...,0.023295,-0.019617,-0.062528,-0.094405,-0.105439,-0.123830,-0.166741,-0.159385,-0.106665,-0.131186
3,0.030618,0.020412,0.058747,0.124712,0.155828,0.161553,0.167777,0.170515,0.165038,0.141141,...,-0.073682,-0.052524,-0.023897,-0.009210,0.016429,0.058000,0.097828,0.114755,0.090360,0.054764
4,-1.071597,-6.912540,-13.677612,-16.463519,-15.790084,-11.653620,-7.668311,-9.942997,-14.321553,-13.704648,...,-5.227723,-6.107613,-6.577051,-6.123589,-5.324806,-4.965969,-4.464579,-3.442138,-3.794831,-6.133420


In [6]:
SVM_oneclass = svm.OneClassSVM(kernel="rbf", gamma=0.001, nu=1)

In [7]:
# Training the algorithm with the features. 
# This stage is very time consuming processes. In my laptop it took more than an hour to train for 200,000 observations. 
# For rbf, the time taken is even more.

SVM_oneclass.fit(data_train) 

OneClassSVM(cache_size=200, coef0=0.0, degree=3, gamma=0.001, kernel='rbf',
            max_iter=-1, nu=1, random_state=None, shrinking=True, tol=0.001,
            verbose=False)

In [8]:
pred_anomaly = SVM_oneclass.predict(data_valid)
#pred_anomaly = SVM_oneclass.predict(X_test)

# predict() : Effectue un ajustement sur X et renvoie les Y pour X.
# Renvoie -1 pour les valeurs aberrantes/outliers et 1 pour les valeurs centrales.

In [10]:
from collections import Counter
Counter(pred_anomaly)

## -1 : anomalies : On en a 581 ?? On ne devrait pas obtenir beaucoup -1, mais juste un petit nombre????

Counter({-1: 594})

In [ ]:
# On va essayer de réduire ce nombre d'anomalies, de -1, en changeant certains paramètres : 
# - kernel
# - gammma
# - nu 

In [ ]:
# Définition des hyperparamètres pour One Class SVM
tb_count_moins1 = pd.DataFrame(index = ['Count outliers'])
tb_count_1 = pd.DataFrame(index = ['Count inliers'])

kernels = ["linear", "poly", "rbf", "sigmoid"]
for kernel in kernels:
    tb_pred = pd.DataFrame()
    SVM_oneclass = svm.OneClassSVM(kernel=kernel, gamma=0.001, nu=0.95)
    SVM_oneclass.fit(data_train) 
    pred_anomaly = SVM_oneclass.predict(data_valid)
    
    tb_count_1[kernel]= [int(Counter(pred_anomaly)[1])]
    tb_count_moins1[kernel]= [int(Counter(pred_anomaly)[-1])]
    
pd.concat([tb_count_1, tb_count_moins1])

#from collections import Counter
#Counter(pred_anomaly)

In [114]:
tb_count_kernel = pd.concat([tb_count_1, tb_count_moins1])

In [115]:
tb_count_kernel.to_csv('Tb count Anomalie par Kernel.csv', sep = ';', index = True, encoding = 'utf-8', na_rep = 'NA')

In [ ]:
# on a le moins d'outliers avec kernel='rbf'

In [121]:
tb_count_moins1 = pd.DataFrame(index = ['Count outliers'])
tb_count_1 = pd.DataFrame(index = ['Count inliers'])

seq_nu = [0.25, 0.5, 0.75, 0.95]
for nu_i in seq_nu:
    tb_pred = pd.DataFrame()
    SVM_oneclass = svm.OneClassSVM(kernel="rbf", gamma=0.001, nu=nu_i)
    SVM_oneclass.fit(data_train) 
    pred_anomaly = SVM_oneclass.predict(data_valid)
    
    tb_count_1[nu_i] = [int(Counter(pred_anomaly)[1])]
    tb_count_moins1[nu_i] = [int(Counter(pred_anomaly)[-1])]
    
tb_count_nu = pd.concat([tb_count_1, tb_count_moins1])
tb_count_nu

In [125]:
tb_count_nu.to_csv('Tb count Anomalie par nu.csv', sep = ';', index = True, encoding = 'utf-8', na_rep = 'NA')

In [126]:
tb_count_moins1 = pd.DataFrame(index = ['Count outliers'])
tb_count_1 = pd.DataFrame(index = ['Count inliers'])

seq_gamma = [0.01, 0.001, 0.0001]
for gamma_i in seq_gamma:
    tb_pred = pd.DataFrame()
    SVM_oneclass = svm.OneClassSVM(kernel="rbf", gamma=gamma_i, nu=0.95)
    SVM_oneclass.fit(data_train) 
    pred_anomaly = SVM_oneclass.predict(data_valid)
    
    tb_count_1[gamma_i] = [int(Counter(pred_anomaly)[1])]
    tb_count_moins1[gamma_i] = [int(Counter(pred_anomaly)[-1])]
    
tb_count_gamma = pd.concat([tb_count_1, tb_count_moins1])
tb_count_gamma

,0.0100,0.0010,0.0001
Count inliers,17,51,52
Count outliers,577,543,542


In [127]:
tb_count_gamma.to_csv('Tb count Anomalie par gamma.csv', sep = ';', index = True, encoding = 'utf-8', na_rep = 'NA')

In [ ]:
## On va utiliser les paramètres qui minimise le nombre de -1 :
# qui sont kernel = 'rbf', gamma= ''?, et nu= ''?

In [7]:
kernel = 'rbf'
gamma_i = 0.0001
nu = 0.25
SVM_oneclass = svm.OneClassSVM(kernel=kernel, gamma=gamma_i, nu=nu)
SVM_oneclass.fit(data_train) 
pred_anomaly = SVM_oneclass.predict(data_valid)

In [9]:
from collections import Counter
Counter(pred_anomaly)
# {1: 279, -1: 315}

Counter({1: 279, -1: 315})

In [ ]:
#mean(pred_anomaly!=Yval)

In [131]:
# score_samples(self, X) : Raw scoring function of the samples.
from statistics import mean
moyenne_scores_ech = mean(SVM_oneclass.score_samples(data_valid))

In [39]:
type(pred_anomaly)

numpy.ndarray

In [ ]:
## Enregistrons nos résultats dans un csv

In [10]:
pred_anomaly2 = pd.DataFrame(pred_anomaly)

In [11]:
pred_anomaly2 = pred_anomaly2.replace(1, 0)
pred_anomaly2 = pred_anomaly2.replace(-1, 1)

In [ ]:
# Sous quel format dois-je envoyer ma prédiction?
# Votre fichier réponse y_test_template doit être un fichier .csv et utiliser l'élément ";" comme séparateur.
#Votre fichier final doit avoir cette forme : 
# seqID;anomaly
# 0;0
# 1;0
# 2;1
# 3;0
# ...;...
# ou : 
# seqID : Numéro de la séquence. (int) 
# prediction : 1 si la séquence est anormale, 0 sinon. (int) 

In [12]:
data = {'seqID': np.arange(0,len(data_valid)), 'anomaly': pred_anomaly2[0]}
y_test_template = pd.DataFrame(data)

In [22]:
y_test_template["seqID"] = y_test_template["seqID"].astype(int)

In [23]:
y_test_template["anomaly"] = y_test_template["anomaly"].astype(int)

In [24]:
# exportation du fichier sous .csv
y_test_template.to_csv('y_test_template.csv',  index = False, sep=";")